# Информация о квестах Москвы

В данной части исследования мы произвели сбор информации о квестах Москвы. 

Сайты: 
- https://mir-kvestov.ru/
- https://topkvestov.ru/

### Описание технической части реализации задачи.

Для сбора и хранения данных мы использовали предназначенные специально для этого библиотеки requests и bs4. Для хранения информации о квестах и более удобной работы с ней мы решили создать модель, которую назвали Quest. С самого начала мы хотели собирать данные сразу с нескольких сайтов, поэтому решили разработать "гибкую" структуру программы: мы создали класс DataCollector, который описывает поведение сборщика данных с определенного сайта и содежит все требования по его интерфейсу и работе. Далее нами были добавлены два класса для сбора данных с конкретных сайтов: MirKvestovDataCollector для сайта Мир Квестов(https://mir-kvestov.ru/) и TopKvestovDataCollector для сайта Топ Квестов(https://topkvestov.ru/). Оба класса наследуют от DataCollector, поэтому мы можем гарантировать их корректную работу. После сбора информации о квестах мы сохранили все полученные данные в файл "quests.xlsx".

### Описание собираемых данных

Мы решили, что наиболее удобным и безопасным будет хранение данных в строковом виде, поэтому использовали тип str в нашей модели. Собирая информацию о квестах, мы искали значения следующих переменных:
- Название квеста
- Компания, организующая квест
- Допустимое число игроков
- Продолжительность квеста в минутах
- Цена квеста
- Возрастные ограничения для игроков
- Уровень сложности квеста
- Контактный телефон
- Адрес локации
- Описание квеста

Во время сбора данных о сложности квеста мы столкнулись с проблемой: на разных сайтах оценка сложности имела разную шкалу. Было принято решение при необходимости приводить данные к унифицированному виду.

### Исходный код

In [ ]:
# Подключение библиотек
import requests
import time as wait
import pandas as pd
from bs4 import BeautifulSoup
from abc import abstractmethod, ABC


In [ ]:
class Quest:
    """
    Представляет модель для хранения информации о квестах после сбора данных с сайтов.
    """

    def __init__(self, quest_name: str, company_name: str,
                 number_of_players: str, time: str, price: str, minimal_age: str,
                 difficulty_level: str, contact_phone: str, address: str, description: str) -> None:
        # Название квеста.
        self.quest_name = quest_name

        # Компания - организатор.
        self.company_name = company_name

        # Допустимое число игроков.
        self.number_of_players = number_of_players

        # Продолжительность квеста.
        self.time = time

        # Цена квеста.
        self.price = price

        # Возрастные ограничения квеста.
        self.minimal_age = minimal_age

        # Уровень сложности квеста.
        self.difficulty_level = difficulty_level

        # Контактный телефон.
        self.contact_phone = contact_phone

        # Адрес.
        self.adress = address

        # Описание.
        self.description = description  
        
    def to_pandas(self) -> list:
        return [self.quest_name, self.company_name, self.number_of_players,
                self.time, self.price, self.minimal_age, self.difficulty_level,
                self.contact_phone, self.adress, self.description]
        
    def __str__(self) -> str:
        return f"{self.quest_name}({self.number_of_players}, {self.time}): {self.contact_phone}, {self.adress}"
    

In [ ]:
class DataCollector(ABC):
    """
    Представляет поведение объекта, собирающего данные о квестах с конкретного сайта.
    """
    
    @abstractmethod
    def get_data(self, page_count: int) -> list:
        """
        Получить данные о квестах.
        :param page_count: максимальное количество страниц, с которых будет произведен сбор данных.  
        :return: Собранные данные в формате Quest.
        """
        pass

In [ ]:
class MirKvestovDataCollector(DataCollector):
    """
    Осуществляет сбор данных с сайта Мир Квестов (https://mir-kvestov.ru/).
    """
    
    def __init__(self):
        self.start_uri = "https://mir-kvestov.ru/"
    
    def __get_links(self):
        page = BeautifulSoup(requests.get(self.start_uri).text, 'html.parser')
        links = page.body.main.findAll("a", class_="quest_tile_name_link")
        
        return [self.start_uri + link["href"] for link in links]
    
    def __get_quest(self, link):
        try:
            page = BeautifulSoup(requests.get(link).text, 'html.parser')
            # Вспомогательные переменные для ограничения области поиска.
            masthead = page.main.find(class_="masthead")
            main_info = masthead.find(class_="params-ul").findAll("li", class_="cell")
            contacts = page.find(class_="contacts", id="contacts")

            quest_name = masthead.findAll("h1")[0].text
            company_name = masthead.find("p", class_="company").text
            number_of_players = main_info[0].find("span", class_="td").get_text().replace("\u200a–\u200a", "-")
            game_time = main_info[1].find("span", class_="td").get_text()
            price = main_info[2].find("span", class_="td").get_text()
            minimal_age = main_info[5].find("span", class_="td").text.replace("*", "")
            difficulty = min(10, int(len(main_info[3].find("span", class_="td").findAll("i", class_="fa fa-key")) / 3 * 10))
            address = contacts.span.span.text
            contact_phone = contacts.a.text
            description = "".join([element.text for element in page.findAll(id="description")[0].findAll("p")])
            
            return Quest(quest_name, company_name,
                         number_of_players, game_time, price, minimal_age, difficulty,
                         contact_phone, address, description)
        except:
            return None

    def get_data(self, page_count: int) -> list:
        data = list()
        for link in self.__get_links()[:page_count]:
            data.append(self.__get_quest(link))
            wait.sleep(1)
        return data

In [ ]:
class TopKvestovDataCollector(DataCollector):
    """
    Осуществляет сбор данных с сайта Топ Квестов (https://topkvestov.ru/).
    """
    
    def __init__(self):
        self.start_uri = "https://topkvestov.ru/"
        
    def __get_links(self):
        page = BeautifulSoup(requests.get(self.start_uri).text, 'html.parser')
        
        return [link["href"] for link in page.find(class_="row-f", id="quests-block").findAll("a", class_="cover-content-hover")]
    
    def __get_quest(self, link):        
        try:
            page = BeautifulSoup(requests.get(link).text, 'html.parser')

            # Вспомогательная переменная для ограничения области поиска.
            contacts = page.find(class_="resp-list")

            quest_name = page.body.find("h1").text
            company_name = page.find(class_="from-company text-pur").a.text
            number_of_players = page.find(class_="human-count").text.strip()
            game_time = page.find(class_="quest-time").text.strip()
            difficulty = page.find(class_="level-count").text.strip()
            minimal_age = page.find(class_="badge badge-lg purple age-limit").text.strip()
            price = page.find(class_="price_value").text.strip()
            contact_phone = contacts.find("a", class_="phone text-yel").text.strip()
            address = contacts.findAll("li")[2].text.replace("(показать на карте)", "")
            description = page.find(class_="quest-text-big open").text.replace("\xa0", "")
            
            return Quest(quest_name, company_name,
                         number_of_players, game_time, price, minimal_age, difficulty,
                         contact_phone, address, description)
        except:
            return None
        
    def get_data(self, page_count: int) -> list:
        data = list()
        for link in self.__get_links()[:page_count]:
            data.append(self.__get_quest(link))
            wait.sleep(1)
        return data

In [ ]:
web_pages_per_collector = 100
output_file = "quests.xlsx"

collectors = [
    MirKvestovDataCollector(),
    TopKvestovDataCollector()
]

quests = list()
for collector in collectors:
    quests.extend(collector.get_data(web_pages_per_collector))

df = pd.DataFrame([x.to_pandas() for x in quests])
df.columns = ["Название", "Организатор", "Количество участников",
              "Продолжительность(мин)", "Цена", "Возрастное ограничение",
              "Сложность", "Телефон", "Адрес", "Описание"]

df.to_excel(output_file)
df


KeyboardInterrupt



Авторы: Волкова Татьяна и Ларина Дарья (БПИ224)